In [3]:
import numpy as np
from numpy.linalg import inv
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from check_inverse import check_inverse_rank

def multi_class(X, y, Xnew, use_poly, lambda_ridge, order):
    
    encoder = OneHotEncoder(sparse_output=False)
    ytr_onehot = encoder.fit_transform(y)
    
    print(f"ytr_onehot:")
    print(ytr_onehot)
    
    # poly
    if use_poly:
        poly = PolynomialFeatures(order)
        P = poly.fit_transform(X)
        P_new = poly.fit_transform(Xnew)
        
        print(f"P with shape {P.shape}:")
        print(P)
        print(f"Pnew with shape {P_new.shape}:")
        print(P_new)

        if (check_inverse_rank(P.T @ P)):
            w_poly = inv(P.T @ P) @ P.T @ ytr_onehot
            print(f"w (poly):")
            print(w_poly)
            
            # predict new input
            yts_pred = P_new @ w_poly
            print(f"pred_y (poly):\n {yts_pred}")
            
            # predict yts_pred_class
            yts_pred_class = np.argmax(yts_pred, axis=1) + 1
            print(f"yts_pred_class (poly):")
            print(yts_pred_class)
        else:
            L2_reg = np.identity(P.shape[1])
            w_poly_ridge = inv(P.T @ P + lambda_ridge * L2_reg) @ P.T @ ytr_onehot
            print(f"w (poly ridge):")
            print(w_poly_ridge)
            
            # predict new input
            yts_pred = P_new @ w_poly_ridge
            print(f"pred_y (poly ridge): {yts_pred}")
            
            # predict yts_pred_class
            yts_pred_class = np.argmax(yts_pred, axis=1) + 1
            print(f"yts_pred_class (poly ridge):")
            print(yts_pred_class)
    # linear 
    else:
        w = inv(X.T @ X) @ X.T @ ytr_onehot
        print(f"w (linear): {w}")
        yts_pred = Xnew @ w
        
        print(f"yts_pred (linear):")
        print(yts_pred)
        
        yts_pred_class = np.argmax(yts_pred, axis=1)
        print(f"yts_pred_class (linear)")
        print(yts_pred_class)


In [5]:
X = np.array([[1,2,3],[4,0,6],[1,1,0],[0,1,2],[5,7,-2],[-1,4,0]])
# X = np.column_stack([np.ones(X.shape[0]),X])
y = np.array([1,1,2,3,2,3]).reshape(-1,1)
X_test = np.array([[1,-2,3]])
# X_test = np.column_stack([np.ones(X_test[0]),X_test])
use_poly = True
lambda_ridge = 0.001
degree = 3
multi_class(X, y, X_test, use_poly, lambda_ridge, degree)

ytr_onehot:
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]
P with shape (6, 20):
[[  1.   1.   2.   3.   1.   2.   3.   4.   6.   9.   1.   2.   3.   4.
    6.   9.   8.  12.  18.  27.]
 [  1.   4.   0.   6.  16.   0.  24.   0.   0.  36.  64.   0.  96.   0.
    0. 144.   0.   0.   0. 216.]
 [  1.   1.   1.   0.   1.   1.   0.   1.   0.   0.   1.   1.   0.   1.
    0.   0.   1.   0.   0.   0.]
 [  1.   0.   1.   2.   0.   0.   0.   1.   2.   4.   0.   0.   0.   0.
    0.   0.   1.   2.   4.   8.]
 [  1.   5.   7.  -2.  25.  35. -10.  49. -14.   4. 125. 175. -50. 245.
  -70.  20. 343. -98.  28.  -8.]
 [  1.  -1.   4.   0.   1.  -4.  -0.  16.   0.   0.  -1.   4.   0. -16.
   -0.  -0.  64.   0.   0.   0.]]
Pnew with shape (1, 20):
[[  1.   1.  -2.   3.   1.  -2.   3.   4.  -6.   9.   1.  -2.   3.   4.
   -6.   9.  -8.  12. -18.  27.]]
matrix rank is : 12
matrix size is : (20, 20)
matrix is not invertible
w (poly ridge):
[[-0.01176707  0.17479266  0.04418068]
 [ 0.